In [1]:
%load_ext autoreload
%autoreload 2
import warnings
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = 'last_expr'

In [22]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
mlflow_artifact_uri = config['mlflow']['target_mlflow_artifact_uri']

In [3]:
import flash
from flash.core.data.utils import download_data

download_data(config['text-example']['download_url'], 'data/')

In [4]:
from flash.text import TextClassificationData, TextClassifier

input_fields,target_fields = 'review', 'sentiment'
batch_size=512
datamodule = TextClassificationData.from_csv(
    train_file=config['text-example']['train_file'],
    val_file=config['text-example']['val_file'],
    test_file=config['text-example']['test_file'],
    input_fields=input_fields,
    target_fields=target_fields,
    batch_size=batch_size
)

Using custom data configuration default
100%|██████████| 23/23 [00:25<00:00,  1.11s/ba]
Using custom data configuration default
100%|██████████| 3/3 [00:02<00:00,  1.14ba/s]
Using custom data configuration default
100%|██████████| 3/3 [00:02<00:00,  1.14ba/s]


In [5]:
model = TextClassifier(num_classes=datamodule.num_classes, backbone='prajjwal1/bert-mini')

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

**Backbone** Can be seen on huggingface: [huggingface-link](https://huggingface.co/prajjwal1/bert-mini)

In [6]:
import os
from pytorch_lightning.loggers import MLFlowLogger

username = os.getenv("uid")
mlf_logger = MLFlowLogger(experiment_name=f'lightning-flash-{username}-tester', tracking_uri=config['mlflow']['mlflow_uri'])

In [7]:
 trainer = flash.Trainer(logger=mlf_logger, max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [8]:
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

Experiment with name lightning-flash-chengb-tester not found. Creating it.

  | Name    | Type                          | Params
----------------------------------------------------------
0 | metrics | ModuleDict                    | 0     
1 | model   | BertForSequenceClassification | 11.2 M
----------------------------------------------------------
514       Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.684    Total estimated model params size (MB)
Epoch 0:  92%|█████████▏| 44/48 [13:07<01:11, 17.89s/it, loss=0.638, v_num=c8b3, val_accuracy=0.504, val_cross_entropy=0.710, train_accuracy_step=0.658, train_cross_entropy_step=0.614]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 48/48 [14:23<00:00, 17.98s/it, loss=0.638, v_num=c8b3, val_accuracy=0.674, val_cross_entropy=0.607, train_accuracy_step=0.662, train_cross_entropy_step=0.617, train_accuracy_epoch=0.585, train_cross_entropy_epoch=0.704]


In [9]:
trainer.test()

Testing: 100%|██████████| 5/5 [00:51<00:00, 10.25s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.670799970626831, 'test_cross_entropy': 0.6051112413406372}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.670799970626831,
  'test_cross_entropy': 0.6051112413406372}]

In [13]:
# If one is not training on linux blade and training on windows laptop
import shutil

file_names = os.listdir()
exp_num = '132'
current_artifact_base_fpath = os.getcwd()
source_dir = f'{current_artifact_base_fpath}'
target_dir = mlflow_artifact_uri
for file_name in file_names:
    if file_name  == exp_num:
        for subfolder in os.listdir(file_name):
            for artifacts in os.listdir(os.path.join(file_name,subfolder)):
                orig_artifacts_loc = os.path.join(os.path.join(os.path.join(source_dir, file_name), subfolder), artifacts)
                # print(orig_artifacts_loc)
                new_artifacts_loc = os.path.join(os.path.join(os.path.join(source_dir, file_name), subfolder), 'artifacts')
                # print(new_artifacts_loc)
                shutil.move(orig_artifacts_loc, new_artifacts_loc)
        shutil.move(os.path.join(source_dir, file_name), target_dir)

#### Take a look at mlflow for best run 

In [19]:
run_id = '518e552a871c46549611d1e6d560c8b3'
model_path = f'{mlflow_artifact_uri}\\{exp_num}\\{run_id}\\artifacts'
for pt in os.listdir(model_path):
    if pt.endswith('.ckpt'):
        ckpt_path = f'{model_path}\\{pt}'

In [20]:
model = TextClassifier.load_from_checkpoint(ckpt_path)

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [25]:
# pred_datamodule = TextClassificationData.from_csv(
#     predict_file=config['text-example']['predict_file'],
#     input_fields=input_fields,
# )
# predictions = flash.Trainer().predict(model, datamodule=pred_datamodule)
# print(predictions)

predictions = model.predict([
    "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
    "I come from Bulgaria where it 's almost impossible to have a tornado."
    "Very, very afraid"
    "This guy has done a great job with this movie!",
])
print(predictions)

['negative', 'positive']
